In [12]:
import pandas as pd 
import numpy as np 


In [13]:
data_elections = pd.read_excel("../data/s_round_elect_results.xls",skiprows=3)

In [14]:
data_elections["LE PEN"] = data_elections["% Voix/Exp.1"]
data_elections["MACRON"] = data_elections["% Voix/Exp"]

In [15]:
data_elections = data_elections[["Libellé de la commune", "Code du département", "Code de la commune", "Inscrits", "% Abs/Ins",\
                                 "LE PEN", "MACRON"]]

In [16]:
data_elections.head()

,Libellé de la commune,Code du département,Code de la commune,Inscrits,% Abs/Ins,LE PEN,MACRON
0,L'Abergement-Clémenciat,1,1,598,16.72,39.96,60.04
1,L'Abergement-de-Varey,1,2,209,15.31,38.00,62.00
2,Ambérieu-en-Bugey,1,4,8586,26.93,41.11,58.89
3,Ambérieux-en-Dombes,1,5,1172,22.10,47.91,52.09
4,Ambléon,1,6,99,23.23,49.15,50.85


In [17]:
data_elections["Code du département"].unique()

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       '2A', '2B', 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 'ZA', 'ZB', 'ZC', 'ZD',
       'ZM', 'ZN', 'ZP', 'ZS', 'ZW', 'ZX', 'ZZ'], dtype=object)

Data type seems to differ inside an unique column ("Code du département"), so we format all data in string

In [18]:
data_elections["Code du département"] = data_elections["Code du département"].map(lambda x : str(x))
data_elections["Code du département"].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '2A', '2B', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
       '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
       '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55',
       '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66',
       '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77',
       '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88',
       '89', '90', '91', '92', '93', '94', '95', 'ZA', 'ZB', 'ZC', 'ZD',
       'ZM', 'ZN', 'ZP', 'ZS', 'ZW', 'ZX', 'ZZ'], dtype=object)

We remove COM related data (St Pierre et Miquelon, Nouvelle-Calédonie, Polynésie, Wallis&Futuna, St-Martin) + french who vote but actual live abroad. Still, we keep data from la Réunion, la Martinique, la Guadeloupe and la Guyanne

In [19]:
masque = data_elections["Code du département"].isin(["ZZ", "ZW", "ZP", "ZM", "ZN" , "ZS", "ZX"])
data_elections = data_elections[~masque]
data_elections["Code du département"].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '2A', '2B', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
       '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
       '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55',
       '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66',
       '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77',
       '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88',
       '89', '90', '91', '92', '93', '94', '95', 'ZA', 'ZB', 'ZC', 'ZD'],
      dtype=object)

Based on "Communes" and "Départements" columns, we calculate the "INSEE" code of a particular city. This will turn to be useful to merge different database in the futur. 

In [20]:
def format_code_commune(x):
    if x < 10 :
        return "00" + str(x)
    if x < 100 :
        return "0" + str(x)
    else :
        return str(x)
def format_code_departement(x):
    if x == '2A' or x == '2B':
        return x
    if x == 'ZA' or x == 'ZB' or x == 'ZC' or x == 'ZD':
        return "97" 
    if int(x) < 10 :
        return "0" + x 
    else :
        return x 

In [21]:
data_elections["Code du département"] = data_elections["Code du département"].apply(format_code_departement)
data_elections["Code de la commune"] = data_elections["Code de la commune"].apply(format_code_commune)
data_elections["Code Insee"] = data_elections["Code du département"] + data_elections["Code de la commune"]

In [22]:
data_elections = data_elections.drop(columns=["Libellé de la commune", "Code du département",\
                                             "Code de la commune"])

In [23]:
data_elections.to_csv("../data/clean_s_round_elect_results.csv", index = False)